In [1]:
import manim as mn
from manim import *

C:\Users\shrey\anaconda3\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [3]:
%%manim -qm -v WARNING PopulationMigration

from manim import *
import numpy as np
import random

class PopulationMigration(Scene):
    def construct(self):
        total_dots = 150  # Total population size
        
        # Transition Probabilities
        prob_A_to_B = 0.2 
        prob_B_to_A = 0.1 
        
        # Column-stochastic transition matrix T
        matrix_vals = np.array([
            [1 - prob_A_to_B, prob_B_to_A],
            [prob_A_to_B,     1 - prob_B_to_A]
        ])

        # Cities (initial positions, then shifted as a group)
        city_a = Circle(radius=1.5, color=BLUE, fill_opacity=0.2).shift(LEFT * 3)
        city_b = Circle(radius=1.5, color=RED, fill_opacity=0.2).shift(RIGHT * 3)
        
        label_a = Text("City A").next_to(city_a, UP)
        label_b = Text("City B").next_to(city_b, UP)
        
        # Transition Arrows
        arrow_ab = CurvedArrow(city_a.get_top(), city_b.get_top(), angle=-TAU/4, color=YELLOW)
        arrow_ba = CurvedArrow(city_b.get_bottom(), city_a.get_bottom(), angle=-TAU/4, color=YELLOW)
        
        cities_group = VGroup(city_a, city_b, label_a, label_b, arrow_ab, arrow_ba)
        cities_group.shift(DOWN * 0.8 + RIGHT * 0.8)
        
        # Matrix Display T
        matrix_tex = MathTex(
            r"T = \begin{bmatrix} "
            f"{matrix_vals[0][0]:.1f}" + r" & " + f"{matrix_vals[0][1]:.1f}" + r" \\ "
            f"{matrix_vals[1][0]:.1f}" + r" & " + f"{matrix_vals[1][1]:.1f}" +
            r" \end{bmatrix}"
        ).to_corner(UL).scale(0.8)
        
        self.play(
            DrawBorderThenFill(city_a), DrawBorderThenFill(city_b),
            Write(label_a), Write(label_b),
            Create(arrow_ab), Create(arrow_ba),
            Write(matrix_tex),
        )

        mult_tex = MathTex("").scale(0.8).next_to(matrix_tex, DOWN, aligned_edge=LEFT, buff=0.4)
        self.play(Write(mult_tex))

        # Random initialization of population.
        count_a = 90
        count_b = total_dots - count_a
        
        # Create dots
        dots_a = VGroup(*[
            Dot(point=self.get_random_point(city_a), color=BLUE_A, radius=0.08)
            for _ in range(count_a)
        ])
        dots_b = VGroup(*[
            Dot(point=self.get_random_point(city_b), color=RED_A, radius=0.08)
            for _ in range(count_b)
        ])
        
        self.play(FadeIn(dots_a), FadeIn(dots_b))
        
        # Display initial counts inside cities
        count_label_a = Integer(count_a).move_to(city_a).scale(1.5).set_z_index(10)
        count_label_b = Integer(count_b).move_to(city_b).scale(1.5).set_z_index(10)

        # Helper to create vector tex [a; b] positioned RIGHT of T
        def get_vector_tex(a, b, t_index):
            tex = MathTex(
                rf"P_{{{t_index}}} = ",
                r"\begin{bmatrix}" + f"{int(a)}" + r" \\ " + f"{int(b)}" + r"\end{bmatrix}"
            ).scale(0.8)
            tex.next_to(matrix_tex, RIGHT, buff=0.7, aligned_edge=UP)
            return tex

        current_t = 0
        current_vector_tex = get_vector_tex(count_a, count_b, current_t)
        
        # Iteration label (top-right)
        step_label = Text(f"Step {current_t}", font_size=24).to_corner(UR)
        
        self.play(
            Write(count_label_a),
            Write(count_label_b),
            Write(current_vector_tex),
            Write(step_label),
        )

        flow_text_ab = MathTex("").scale(0.6)
        flow_text_ba = MathTex("").scale(0.6)

        flow_text_ab.next_to(current_vector_tex, RIGHT, buff=0.5)
        flow_text_ba.next_to(flow_text_ab, DOWN, aligned_edge=LEFT, buff=0.1)

        self.play(Write(flow_text_ab), Write(flow_text_ba))

        # State vector (as floats for matrix multiplication)
        state_vec = np.array([count_a, count_b], dtype=float)
        
        # Target steady-state vector [50, 100]
        target_vec = np.array([50, 100], dtype=float)

        # Helper for multiplication text: T · [old] = [expected new]
        def get_mult_tex(vec_before, vec_expected):
            a, b = vec_before
            a2, b2 = vec_expected
            tex = MathTex(
                r"T \cdot ",
                r"\begin{bmatrix}" + f"{int(a)}" + r" \\ " + f"{int(b)}" + r"\end{bmatrix}",
                r" = ",
                r"\begin{bmatrix}" + f"{int(a2)}" + r" \\ " + f"{int(b2)}" + r"\end{bmatrix}",
            ).scale(0.8)
            tex.next_to(matrix_tex, DOWN, aligned_edge=LEFT, buff=0.4)
            return tex

        # SIMULATION LOOP
        steps = 15  # Number of time steps
        
        for i in range(steps):
            # Compute the expected next state by matrix multiplication (for display)
            expected_next_real = matrix_vals @ state_vec  # expected real-valued result

            if i == steps - 1:
                expected_next_real = target_vec.copy()
            
            expected_next_rounded = np.rint(expected_next_real).astype(int)

            # Determine number of movers in each direction (probabilistic flows)
            num_moving_ab = int(len(dots_a) * prob_A_to_B)  # from A -> B
            num_moving_ba = int(len(dots_b) * prob_B_to_A)  # from B -> A

            # Ensure we don't try to move more dots than available
            num_moving_ab = min(num_moving_ab, len(dots_a))
            num_moving_ba = min(num_moving_ba, len(dots_b))

            # Update flow lines beside P_t
            new_flow_ab = MathTex(
                rf"{num_moving_ab}\ \text{{move A $\to$ B}}"
            ).scale(0.6)
            new_flow_ba = MathTex(
                rf"{num_moving_ba}\ \text{{move B $\to$ A}}"
            ).scale(0.6)
            
            # re-position relative to updated P_t text
            new_flow_ab.next_to(current_vector_tex, RIGHT, buff=0.5)
            new_flow_ba.next_to(new_flow_ab, DOWN, aligned_edge=LEFT, buff=0.1)
            
            self.play(
                Transform(flow_text_ab, new_flow_ab),
                Transform(flow_text_ba, new_flow_ba),
                run_time=0.3
            )

            # Select movers
            movers_ab = dots_a[-num_moving_ab:] if num_moving_ab > 0 else VGroup()
            movers_ba = dots_b[-num_moving_ba:] if num_moving_ba > 0 else VGroup()

            # Keepers
            keep_a = dots_a[:-num_moving_ab] if num_moving_ab > 0 else dots_a
            keep_b = dots_b[:-num_moving_ba] if num_moving_ba > 0 else dots_b

            # Build animations and move both sets of movers along their arrows
            anims = []
            if len(movers_ab) > 0:
                anims.append(
                    AnimationGroup(
                        *[MoveAlongPath(d, arrow_ab, run_time=1.5, rate_func=linear) for d in movers_ab],
                        lag_ratio=0.02
                    )
                )
            if len(movers_ba) > 0:
                anims.append(
                    AnimationGroup(
                        *[MoveAlongPath(d, arrow_ba, run_time=1.5, rate_func=linear) for d in movers_ba],
                        lag_ratio=0.02
                    )
                )
            if anims:
                self.play(*anims)

            # Settle moved dots into new cities (random locations inside each circle)
            settle_anims = []
            new_dots_a_from_b = movers_ba.copy()
            new_dots_b_from_a = movers_ab.copy()

            new_dots_a_from_b.set_color(BLUE_A)
            new_dots_b_from_a.set_color(RED_A)

            for d in new_dots_a_from_b:
                settle_anims.append(d.animate.move_to(self.get_random_point(city_a)))
            for d in new_dots_b_from_a:
                settle_anims.append(d.animate.move_to(self.get_random_point(city_b)))

            if settle_anims:
                self.play(*settle_anims, run_time=0.8)

            # Rebuild dot groups after movements
            dots_a = VGroup(*keep_a, *new_dots_a_from_b)
            dots_b = VGroup(*keep_b, *new_dots_b_from_a)

            if i == steps - 1:
                final_needed = target_vec.astype(int)
                current_counts = np.array([len(dots_a), len(dots_b)], dtype=int)
                diff = final_needed - current_counts
                # If A needs more, move from B -> A; if needs fewer, move from A -> B
                if diff[0] > 0:
                    # move diff[0] dots from B to A
                    to_move = min(diff[0], len(dots_b))
                    movers = dots_b[-to_move:] if to_move > 0 else VGroup()
                    keep_b = dots_b[:-to_move] if to_move > 0 else dots_b
                    for d in movers:
                        # animate along arrow then settle
                        self.play(MoveAlongPath(d, arrow_ba, run_time=0.6), d.animate.move_to(self.get_random_point(city_a)))
                    dots_a = VGroup(*dots_a, *movers)
                    dots_b = VGroup(*keep_b)
                elif diff[0] < 0:
                    to_move = min(-diff[0], len(dots_a))
                    movers = dots_a[-to_move:] if to_move > 0 else VGroup()
                    keep_a = dots_a[:-to_move] if to_move > 0 else dots_a
                    for d in movers:
                        self.play(MoveAlongPath(d, arrow_ab, run_time=0.6), d.animate.move_to(self.get_random_point(city_b)))
                    dots_b = VGroup(*dots_b, *movers)
                    dots_a = VGroup(*keep_a)

            # Update displayed counts & texts
            state_vec_before = state_vec.copy()
            state_vec = np.array([len(dots_a), len(dots_b)], dtype=float)  # actual integer counts after moves

            # Update city count labels
            self.play(
                count_label_a.animate.set_value(int(state_vec[0])),
                count_label_b.animate.set_value(int(state_vec[1])),
                run_time=0.5
            )

            # update multiplication text (showing expected/rounded result)
            mult_display = get_mult_tex(state_vec_before, expected_next_rounded)
            self.play(Transform(mult_tex, mult_display), run_time=0.6)

            # update P_t (positioned right of T)
            current_t += 1
            new_vector_tex = get_vector_tex(state_vec[0], state_vec[1], current_t)
            self.play(Transform(current_vector_tex, new_vector_tex), run_time=0.6)

            # update iteration label
            new_step_label = Text(f"Step {current_t}", font_size=24).to_corner(UR)
            self.play(Transform(step_label, new_step_label), run_time=0.4)

            self.wait(0.2)

        self.wait(1.5)

    def get_random_point(self, circle):
        # Helper to get a random point inside a circle
        r = circle.radius * np.sqrt(random.random())
        theta = random.random() * 2 * PI
        return circle.get_center() + np.array([r * np.cos(theta), r * np.sin(theta), 0])


Manim Community v0.19.0